In [16]:
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier 
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

bk = pd.read_csv("/Users/darshmac/Documents/cdac/For DBDA/PML/Cases/Bankruptcy/Bankruptcy.csv", index_col=0)
bk

,D,YR,R1,R2,R3,R4,R5,R6,R7,R8,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
NO,,,,,,,,,,,,,,,,,,,,,
1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,10.36,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,3.13,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,2.41,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07
4,0,80,0.07,0.03,0.04,0.04,0.04,0.06,0.06,5.55,...,-0.02,0.01,0.02,0.02,5.36,1.30,1.12,-0.06,-0.08,-0.09
5,0,81,0.09,0.02,0.03,0.04,0.06,0.08,0.11,2.85,...,0.02,0.07,0.10,0.14,7.74,1.48,1.41,0.03,0.04,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,1,77,0.27,0.03,0.07,0.10,0.09,0.18,0.26,7.58,...,0.11,0.06,0.12,0.17,7.56,2.07,1.45,0.06,0.13,0.19
129,1,77,0.32,0.03,0.03,0.09,0.05,0.06,0.16,3.24,...,0.17,0.07,0.09,0.26,5.99,1.27,2.74,0.06,0.08,0.21
130,1,78,0.08,0.01,0.02,0.05,0.04,0.07,0.15,3.82,...,0.19,0.07,0.12,0.26,7.14,1.89,2.10,0.07,0.12,0.26


In [17]:
#dum_hr = pd.get_dummies(hr, drop_first=True)
X = bk.drop(['D','YR'], axis=1)
y = bk['D']
#we will tune model lets not train test split
#X_train, X_test, y_train, y_test = train_test_split(X,y, 
#                               test_size=0.3,
#                              stratify=y,
#                               random_state=23)

In [18]:
#lets choose different model
lr = LogisticRegression()
svm = SVC(kernel='linear')
dtc = DecisionTreeClassifier(random_state=23)
rf = RandomForestClassifier(random_state=23,
                            n_estimators=25)

In [19]:
#creating stack for models to fit
stack = StackingClassifier(estimators=[('LR', lr),
                                       ('SVM', svm),
                                       ('TREE', dtc)],
                           final_estimator=rf)

#stack.fit(X_train, y_train) # all train set operations
#y_pred = stack.predict(X_test) # all test operations
#print(accuracy_score(y_test, y_pred))

In [20]:
stack.get_params()

{'cv': None,
 'estimators': [('LR', LogisticRegression()),
  ('SVM', SVC(kernel='linear')),
  ('TREE', DecisionTreeClassifier(random_state=23))],
 'final_estimator__bootstrap': True,
 'final_estimator__ccp_alpha': 0.0,
 'final_estimator__class_weight': None,
 'final_estimator__criterion': 'gini',
 'final_estimator__max_depth': None,
 'final_estimator__max_features': 'sqrt',
 'final_estimator__max_leaf_nodes': None,
 'final_estimator__max_samples': None,
 'final_estimator__min_impurity_decrease': 0.0,
 'final_estimator__min_samples_leaf': 1,
 'final_estimator__min_samples_split': 2,
 'final_estimator__min_weight_fraction_leaf': 0.0,
 'final_estimator__n_estimators': 25,
 'final_estimator__n_jobs': None,
 'final_estimator__oob_score': False,
 'final_estimator__random_state': 23,
 'final_estimator__verbose': 0,
 'final_estimator__warm_start': False,
 'final_estimator': RandomForestClassifier(n_estimators=25, random_state=23),
 'n_jobs': None,
 'passthrough': False,
 'stack_method': 'auto'

In [21]:
params = {'LR__C':[0.1,0.5,1,2],
         'TREE__max_depth':[None,3,5],
         'final_estimator__max_features':[2,5,10],
         'passthrough':[True,False]}

In [22]:
from sklearn.model_selection import GridSearchCV,KFold

In [12]:
kflod = KFold(n_splits=5, shuffle=True,random_state=23)

In [26]:
gcv = GridSearchCV(stack,param_grid=params,cv=kflod,verbose=3,scoring='neg_log_loss')

In [ ]:
gcv.fit(X,y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=True;, score=-0.397 total time=   0.3s
[CV 2/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=True;, score=-0.453 total time=   0.3s


/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=True;, score=-0.455 total time=   0.4s
[CV 4/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=True;, score=-0.343 total time=   0.3s
[CV 5/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=True;, score=-0.365 total time=   0.3s
[CV 1/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=False;, score=-4.499 total time=   0.3s
[CV 2/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=False;, score=-1.760 total time=   0.3s


/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=False;, score=-1.584 total time=   0.4s
[CV 4/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=False;, score=-1.656 total time=   0.3s
[CV 5/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=2, passthrough=False;, score=-0.412 total time=   0.3s
[CV 1/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=5, passthrough=True;, score=-0.443 total time=   0.3s
[CV 2/5] END LR__C=0.1, TREE__max_depth=None, final_estimator__max_features=5, passthrough=True;, score=-1.733 total time=   0.3s


...

[CV 3/5] END LR__C=2, TREE__max_depth=5, final_estimator__max_features=2, passthrough=False;, score=-1.638 total time=   0.4s


/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

[CV 4/5] END LR__C=2, TREE__max_depth=5, final_estimator__max_features=2, passthrough=False;, score=-1.658 total time=   0.4s


/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/darshmac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

[CV 5/5] END LR__C=2, TREE__max_depth=5, final_estimator__max_features=2, passthrough=False;, score=-1.744 total time=   0.4s


In [25]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__C': 0.5, 'TREE__max_depth': 3, 'final_estimator__max_features': 10, 'passthrough': True}
0.849002849002849
